In [81]:
import pandas
import requests
import datetime
import time
import json

In [82]:
etherscan_api_key = "SU775DFF8GEGMER3B8ZPK3QZIJWGIBE45T"

In [83]:
##https://api.etherscan.io/api?module=block&action=getblockreward&blockno=2165403&apikey=YourApiKeyToken
url_endpoint = "https://api.etherscan.io/api"
params = {"module" : "block", "action" : "getblockreward", "blockno" : "2165403", "apikey" : etherscan_api_key }
resp = requests.get(url_endpoint, params=params)

In [84]:
##Getting the most recent block number
params = {"module": "proxy", "action":"eth_blockNumber", "apikey": etherscan_api_key}
resp = requests.get(url_endpoint, params=params)
resp_dict = json.loads(resp.content) #Get a string, must eval to a dict
latest_block = int(resp_dict['result'], 0) #Get it in Hex, need to eval to int
latest_block

4836927

In [85]:
#Starting Point
starting_block = 4830000 #picked a solid starting point
hex_starting_block = hex(starting_block)
hex_starting_block

'0x49b330'

In [86]:
#Get Transactions in a Block
#https://github.com/ethereum/wiki/wiki/JSON-RPC#eth_gettransactionbyhash
params = {"module":"proxy", "action":"eth_getBlockByNumber", "tag" : hex_starting_block, "boolean":"true", "apikey": etherscan_api_key}
resp = requests.get(url_endpoint, params=params)
resp_dict = json.loads(resp.content) #Get a string, must eval to a dict

In [87]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status

In [88]:
#Transactions processed for this block
processed_txs = []

#Timestamp

timestamp = int(resp_dict['result']['timestamp'],0)
timestamp = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

#Every other feature will be found in the transactions

transactions = resp_dict['result']['transactions']
transactions

for i in transactions:
    #Initialize the next transaction
    next_tx = {'datetime': timestamp, 'network': "Ethereum"}
    
    #Unique Hash
    next_tx['txhash'] = i['hash']
    
    #Account Sending
    next_tx['sender'] = i['from']
    
    #Account Receiving
    next_tx['receiver'] = i['to']
    
    #Tx Fee
    #Can't evaluate the fee until we get the receipt info, which is a separate call.
    #Oh Bother.
    next_tx['gasprice'] = int(i['gasPrice'],0)    
    
    processed_txs.append(next_tx)

In [94]:
#Now I'm going through and grabbing the transaction hashes
for i in processed_txs:
    tx_hash = i['txhash']
    
    params = {"module":"proxy", "action":"eth_getTransactionReceipt", "txhash" : tx_hash, "apikey": etherscan_api_key}
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content)
    
    i["gasused"] = int(resp_dict['result']['gasUsed'],0)
    i['status'] = int(resp_dict['result']['status'],0)
    
    time.sleep(.2)

In [104]:
def tx_cost_in_ETH(gasused, gasprice):
    wei_in_ETH = 1000000000000000000
    cost_in_eth = (gasused * gasprice)/float(wei_in_ETH)
    return cost_in_eth

In [106]:
for i in processed_txs:
    i['txfee'] = tx_cost_in_ETH(i['gasused'], i['gasprice'])

In [107]:
processed_txs

[{'datetime': '2017-12-31 07:59:48',
  'gasprice': 91000000000L,
  'gasused': 21000,
  'network': 'Ethereum',
  'receiver': u'0xeddea3a2277e9534d0c4c7951cda819a458f0546',
  'sender': u'0x22b84d5ffea8b801c0422afe752377a64aa738c2',
  'status': 1,
  'txfee': 0.001911,
  'txhash': u'0xea5e2cd6f18336095134d991ff2bc7d0acf7812d0612b32277b1fe96b8d423be'},
 {'datetime': '2017-12-31 07:59:48',
  'gasprice': 68000000000L,
  'gasused': 21000,
  'network': 'Ethereum',
  'receiver': u'0xca6fc96ce8cc0df149abb9496ab63b9bb083d545',
  'sender': u'0x0fd081e3bb178dc45c0cb23202069dda57064258',
  'status': 1,
  'txfee': 0.001428,
  'txhash': u'0x7ccc7a7b7fd80ca7fd90a9c9c1729a04b7afd9127f5d2a3b28143a58a35212d2'},
 {'datetime': '2017-12-31 07:59:48',
  'gasprice': 60000000000L,
  'gasused': 21000,
  'network': 'Ethereum',
  'receiver': u'0x1bc0c0020810c624a68794c4f195b14694cecf5d',
  'sender': u'0x80e29acb842498fe6591f020bd82766dce619d43',
  'status': 1,
  'txfee': 0.00126,
  'txhash': u'0xba9a6026f6d745bbc55